In [ ]:
from pyspark.sql import SparkSession

# Inicializa uma sessão Spark
spark = SparkSession.builder.appName("CidadesNorte").getOrCreate()



In [ ]:
# Lista de cidades do Norte do Brasil e o número de habitantes
dados = [
    ("Manaus", 2219580),
    ("Belém", 1492745),
    ("Porto Velho", 529544),
    ("Boa Vista", 419652),
    ("Palmas", 306296),
    ("Rio Branco", 413418),
    ("Macapá", 503327),
    ("Santarém", 294447),
]

# Define o schema para o DataFrame
schema = ["Cidade", "Habitantes"]

# Cria o DataFrame
df = spark.createDataFrame(dados, schema)

# Mostra o DataFrame
df.show()

In [ ]:
#Criando nossa primeira tabela delta
df.write.format("delta").save("s3a://lakehouse-demo/cidades_norte")

In [ ]:
#Optimizando a tabela
from delta import DeltaTable

delta_table = DeltaTable.forPath(spark, "s3a://lakehouse-demo/cidades_norte")
delta_table.optimize().executeCompaction()

In [ ]:
#Versionamento
delta_table.history().show()

In [ ]:
#Vacuum

delta_table.vacuum(0)

In [ ]:
spark = SparkSession.builder.appName("CidadesNorte")\
                             .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")\
                             .getOrCreate()

In [ ]:

#Vacuum
delta_table.vacuum(0)